In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm
import re
pd.set_option('display.max_columns', 999)

#### Connect to SQLite database

In [2]:
db = sqlite3.connect('data/NPPES_Data_Dissemination_February_2021/npidata2.sqlite')

#### SQL query for npi data

In [7]:
query = """
        SELECT * 
        FROM npidata
        LIMIT 5
 """

#### Assign query to variable

In [8]:
npi_sqlite = pd.read_sql(query, db) 

#### View npi data variable

In [9]:
npi_sqlite

,NPI,Entity_Type_Code,Provider_Organization_Name,Provider_Last_Name,Provider_First_Name,First_Line_Address,Second_Line_Address,Address_City,Address_Postal_Codes,Provider_Credential_Text,Gender,Healthcare_Provider_Taxonomy_Code
0,1841293891,1.0,None,GIBBS,ELMER,49 CLEVELAND ST 310,None,CROSSVILLE,38555,M.D.,M,208600000X
1,1659374601,1.0,None,OBERDICK,WENDY,105 W STONE DR,STE 1F,KINGSPORT,37660,MD,F,207Q00000X
2,1134122187,1.0,None,RUDNICKE,CHERYL,250 25TH AVE N,STE 412,NASHVILLE,37203,CRNP,F,363L00000X
3,1003819046,1.0,None,NYLANDER,BARBARA,345 23RD AVE N,SUITE 209,NASHVILLE,37203,M.D.,F,207VG0400X
4,1750384780,1.0,None,PERRIGIN,JULIE,219 CHURCH ST,None,DICKSON,37055,MD,F,207Q00000X


#### Run query to join npi data, CBSA data and Taxonomy classification with filter on General Acute Care Hospitals, entity type 2 and CBSA

In [ ]:
query1 = """
        SELECT *
        FROM npidata AS n
        INNER JOIN taxonomy AS t
        on n.Healthcare_Provider_Taxonomy_Code = t.Code
        INNER JOIN hops AS h 
        on n.NPI = h.to_npi
        INNER JOIN CBSA as c
        on n.Address_Postal_Codes = c.ZIP
        WHERE Entity_Type_Code = 2.0 AND Classification = 'General Acute Care Hospital'AND CBSA = 34980  
 """

#### Assign variable to above query 

In [ ]:
npi_sqlite1 = pd.read_sql(query1, db)

#### View query

In [ ]:
npi_sqlite1

#### Run query to join npi data, CBSA data and Taxonomy classification with filter on entity type 1 and CBSA

In [ ]:
query2 = """
        SELECT *
        FROM npidata AS n
        INNER JOIN taxonomy AS t
        on n.Healthcare_Provider_Taxonomy_Code = t.Code
        INNER JOIN hops AS h 
        on n.NPI = h.from_npi
        INNER JOIN CBSA as c
        on n.Address_Postal_Codes = c.ZIP
        WHERE Entity_Type_Code = 1.0 AND CBSA = 34980  
 """

#### Assign variable to above query 

In [ ]:
from npi = type code 1, to npi = type code 2

#### View query above

In [ ]:
npi_sqlite2

In [ ]:
query3 = """
 SELECT h.from_npi,h2.to_npi,Entity_Type_Code
        FROM npidata AS n
        INNER JOIN hops AS h 
        on n.NPI = h.from_npi
        INNER JOIN hops AS h2 
        on n.NPI = h2.to_npi
"""

In [ ]:
npi_sqlite3 = pd.read_sql(query3, db)

In [ ]:
npi_sqlite3

In [27]:
query4 = """
    WITH npi_entity_type_1 AS (
        SELECT NPI AS Phys_NPI,
        n.Provider_Organization_Name AS Phys_Office,
        n.Provider_Last_Name AS Phys_Last_Name, 
        n.Provider_First_Name AS Phys_First_Name, 
        t.Classification AS Phys_Class, 
        t.Specialization AS Phys_Spec
        FROM npidata as n
        INNER JOIN CBSA as c
        ON n.Address_Postal_Codes = c.ZIP
        INNER JOIN taxonomy as t
        ON n.Healthcare_Provider_Taxonomy_Code = t.Code
        WHERE Entity_Type_Code = 1.0 AND CBSA = 34980
    ), npi_entity_type_2 AS (
        SELECT NPI AS Hosp_NPI, 
        n.Provider_Organization_Name AS Hospital,
        t.Classification AS Hosp_Class, 
        t.Specialization AS Hosp_Spec
        FROM npidata as n
        INNER JOIN CBSA as c
        ON n.Address_Postal_Codes = c.ZIP
        INNER JOIN taxonomy as t
        ON n.Healthcare_Provider_Taxonomy_Code = t.Code
        WHERE Entity_Type_Code = 2.0 AND CBSA = 34980 AND Classification = 'General Acute Care Hospital'
    )
    SELECT from_npi, 
    to_npi, 
    patient_count, 
    transaction_count,
    average_day_wait, 
    Phys_NPI,
    Phys_Office,
    Phys_Last_Name, 
    Phys_First_Name, 
    Phys_Class,
    Phys_Spec,
    Hosp_NPI,
    Hospital,
    Hosp_Class,
    Hosp_Spec
    FROM hops
    INNER JOIN npi_entity_type_1
    ON hops.from_npi = npi_entity_type_1.Phys_NPI
    INNER JOIN npi_entity_type_2
    ON hops.to_npi = npi_entity_type_2.Hosp_NPI  
    WHERE from_npi IN (SELECT Phys_NPI FROM npi_entity_type_1)
    AND to_npi IN (SELECT Hosp_NPI FROM npi_entity_type_2)
"""

In [28]:
filtered_hop_teaming = pd.read_sql(query4, db)

In [6]:
filtered_hop_teaming.shape

NameError: name 'filtered_hop_teaming' is not defined

In [30]:
filtered_hop_teaming.to_csv("data/Filtered_Hop_Team_Data")

In [7]:
filtered_hop_teaming.to_sql('hop_filtered', db, if_exists = 'append', index = False)

In [8]:
db.close()

In [ ]:
query1 = """
        SELECT *
        FROM npidata AS n
        INNER JOIN taxonomy AS t
        on n.Healthcare_Provider_Taxonomy_Code = t.Code
        INNER JOIN hop_filtered AS h 
        on n.NPI = h.to_npi
        INNER JOIN CBSA as c
        on n.Address_Postal_Codes = c.ZIP
        WHERE CBSA = 34980 AND Classification = 'General Acute Care Hospital'
 """

In [10]:
query5 = """
    WITH npi_entity_type_1 AS (
        SELECT NPI AS Phys_NPI,
        n.Provider_Organization_Name AS Phys_Office,
        n.Provider_Last_Name AS Phys_Last_Name, 
        n.Provider_First_Name AS Phys_First_Name, 
        t.Classification AS Phys_Class, 
        t.Specialization AS Phys_Spec,
        n.First_Line_Address AS Phys_Address,
        n.Address_City AS Phys_City,
        n.Address_Postal_Codes AS Phys_Zipcode
        FROM npidata as n
        INNER JOIN CBSA as c
        ON n.Address_Postal_Codes = c.ZIP
        INNER JOIN taxonomy as t
        ON n.Healthcare_Provider_Taxonomy_Code = t.Code
        WHERE Entity_Type_Code = 1.0 AND CBSA = 34980
    ), npi_entity_type_2 AS (
        SELECT NPI AS Hosp_NPI, 
        n.Provider_Organization_Name AS Hospital,
        t.Classification AS Hosp_Class, 
        t.Specialization AS Hosp_Spec
        FROM npidata as n
        INNER JOIN CBSA as c
        ON n.Address_Postal_Codes = c.ZIP
        INNER JOIN taxonomy as t
        ON n.Healthcare_Provider_Taxonomy_Code = t.Code
        WHERE Entity_Type_Code = 2.0 AND CBSA = 34980
    )
    SELECT from_npi, 
    to_npi, 
    patient_count, 
    transaction_count,
    average_day_wait, 
    Phys_NPI,
    Phys_Office,
    Phys_Last_Name, 
    Phys_First_Name, 
    Phys_Class,
    Phys_Spec,
    Phys_Address,
    Phys_City,
    Phys_Zipcode,
    Hosp_NPI,
    Hospital,
    Hosp_Class,
    Hosp_Spec
    FROM hops
    INNER JOIN npi_entity_type_1
    ON hops.from_npi = npi_entity_type_1.Phys_NPI
    INNER JOIN npi_entity_type_2
    ON hops.to_npi = npi_entity_type_2.Hosp_NPI  
    WHERE from_npi IN (SELECT Phys_NPI FROM npi_entity_type_1)
    AND to_npi IN (SELECT Hosp_NPI FROM npi_entity_type_2)
"""

In [11]:
npi_sqlite5 = pd.read_sql(query5, db)

In [12]:
npi_sqlite5.to_csv("./data/list for addresses")

Questions to consider with the data:
- How do we combine and filter datasets to have npi_from to be entity type 1's only and to_npi to be entity type 2's only
- Once we have the completed dataset, Are there any filters we want to put on taxonomy classification types (i.e. drama therapists, veterinarian, bus, train etc.)
- What hospitals are we considering in this analysis?